In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('C:/Users/Dell/Documents/book/unsuper/Projects/first/mental-heath-in-tech-2016_20161114.csv')
df.columns

# Due to my study focusing on employed individuals, I excluded information pertaining to self-employed individuals.

df.shape


get_nan_info = df[df['Are you self-employed?']== 1]


get_nan_info


# get index of above row and then remove them



get_index_self_employed = df[df['Are you self-employed?'] == 1].index


df = df.drop(get_index_self_employed, axis = 0)


df.shape

df_prep = df


# I chose to eliminate the "Country Live" and "State Live" as I am solely interested in the location where each participant is employed,
# and also removed the "why not" columns as they cannot be encoded.
df_prep.drop(['What country do you live in?'],axis = 1,inplace=True)
df_prep.drop(['Why or why not?'],axis = 1,inplace=True)
df_prep.drop(['Why or why not?.1'],axis = 1,inplace=True)


# Dealing with Gender
df['What is your gender?'].unique()


male = ["Male", "male", "Male ", "M", "m", "man", "Cis male", "Male.",
        "Male (cis)", "Man", "Sex is male", "cis male", "Malr", "Dude",
        "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
        "mail", "M|", "male ", "Cis Male", "cisdude", "cis man", "MALE", "Male (trans, FtM)", "Male/genderqueer"]
female = ["Female", "female", "I identify as female.", "female ",
         "Female assigned at birth ", "F", "Woman", "fm", "f", "Cis female",
         "Transitioned, M2F", "Female or Multi-Gender Femme", "Female ",
         "woman", "female/woman", "Cisgender Female", "mtf", "fem",
         "Female (props for making this a freeform field, though)",
         " Female", "Cis-woman", "AFAB", "Transgender woman", "Cis female "]
other = ["Bigender", "non-binary", "Genderfluid (born female)",
        "Other/Transfeminine", "Androgynous", "male 9:1 female, roughly",
        "nb masculine", "genderqueer", "Human", "Genderfluid", "Enby",
        "genderqueer woman", "Queer", "Agender", "Fluid", "Genderflux demi-girl",
        "female-bodied; no feelings about gender", "non-binary", "Male/genderqueer",
        "Nonbinary", "Other", "none of your business", "Unicorn", "human", "Genderqueer"]


df_prep['What is your gender?'] = df_prep['What is your gender?'].replace(male,'Male')


df_prep['What is your gender?'] = df_prep['What is your gender?'].replace(female,'Female')


df_prep['What is your gender?'] = df_prep['What is your gender?'].replace(other,'other')


df['What is your gender?'].unique()


c = df['What is your gender?'].mode()
df_prep['What is your gender?'] = df_prep['What is your gender?'].fillna(c[0])


# Dealing with age

# Checking if there are any NaN values in the age column and then dividing people into five groups
print(df["What is your age?"].isnull().sum())


median_age = df_prep[(df_prep['What is your age?'] >= 18) | (df_prep['What is your age?'] <= 75)]['What is your age?'].median()

print("Median Age is :", median_age)

df_prep['What is your age?'].replace(to_replace = df_prep[(df_prep['What is your age?'] < 18) | (df_prep['What is your age?'] > 75)]['What is your age?'].to_list(), value = median_age, inplace = True)



df['What is your age?'].unique()



# Dealing with total employee. 
# dividing employees into five groups

ordinal_orderings = [
    ['1-5', '6-25', '26-100', '100-500', '500-1000', 'More than 1000']
]
ordinal_features = [
    'How many employees does your company or organization have?'
]
def ordinal_encode(df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: (ordering.index(x)+1))
    return df
df_prep = ordinal_encode(
    df_prep, columns=ordinal_features,orderings=ordinal_orderings
)


df['How many employees does your company or organization have?'].unique()


# Dealing with position


df_prep['Which of the following best describes your work position?'].replace(['Back-end Developer|Front-end Developer', 'Front-end Developer|Back-end Developer'], 'Full track', inplace=True)
tech = ['Developer', 'Full Track',' DevOps', 'SysAdmin', 'Designer', 'Dev Evangelist']
df_prep['Is your primary role within your company related to tech/IT?' ] = np.where((x in df['Which of the following best describes your work position?'] for x in tech) and (df['Is your primary role within your company related to tech/IT?'].isnull()), 1.0, 0.0)
df_prep['Is your primary role within your company related to tech/IT?' ].value_counts(dropna=False)


df['Does your employer provide mental health benefits as part of healthcare coverage?'].unique()


# The columns in this list constitute a cluster of inquiries linked 
# to past employer. These characteristics exhibit minimal correlation with the present study.
# Consequently, I have opted to exclude said columns from the ongoing analysis.
print(df.shape)


this_list = ["Have your previous employers provided mental health benefits?",
            "Were you aware of the options for mental health care provided by your previous employers?",
            "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?",
            "Did your previous employers provide resources to learn more about mental health issues and how to seek help?",
            "Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?",
            "Do you think that discussing a mental health disorder with previous employers would have negative consequences?",
            "Do you think that discussing a physical health issue with previous employers would have negative consequences?",
            "Would you have been willing to discuss a mental health issue with your previous co-workers?",
            "Would you have been willing to discuss a mental health issue with your direct supervisor(s)?",
            "Did you feel that your previous employers took mental health as seriously as physical health?",
            "Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?"]


y = df["Have your previous employers provided mental health benefits?"].value_counts(dropna = False)
print(y)


z =  df["Have your previous employers provided mental health benefits?"]



t = df[z.isnull()]


t[this_list]


# Due to the abundance of missing values, I chose to eliminate those columns. This decision was made not solely due to the presence of missing values,
# but also because those columns are associated with a former employer.


df_prep = df_prep.drop(labels= this_list, axis=1)
df_prep.shape



df['If yes, what condition(s) have you been diagnosed with?'].value_counts(dropna= False)



# Dealing with nan values
# I remove columns with more than 50% of nan values
half_empty_columns = []
#go through every column 
columns = df_prep.columns.tolist()
for i in columns:
    if sum(pd.isnull(df_prep[i])) > (len(df_prep)/2):
        half_empty_columns.append(i) 



half_empty_columns


for i in half_empty_columns:
    df_prep.drop([i],  axis=1, inplace=True)



df_prep.drop('What US state or territory do you live in?',  axis=1, inplace=True)


# Dividing states based on geographical location.
def create_state_encoding(state):
   west = ["Wyoming", "Nevada", "Utah", "California", "Hawaii"]
   northeast = ["Maine", "New Hampshire", "Vermont", "Massachusetts", "Rhode Island",
               "Connecticut", "New York", "Pennsylvania", "New Jersey", "Ohio"]
   east = [ "Delaware","Kentucky", "Tennessee", "North Carolina", "West Virginia", "Virginia","District of Columbia","Maryland"]
   south = ["New Mexico", "Mississippi", "Alabama", "Texas", "Arkansas", "Louisiana", "Arizona"]
   center = ["Illinois", "Indiana","South Dakota","Nebraska","Kansas", "Iowa","Colorado", "Oklahoma"]
   north = ["North Dakota", "Minnesota","Missouri","Wisconsin","Michigan"]
   north_west = ["Idaho", "Montana","Alaska","Washington","Oregon"]
   south_east = ["South Carolina","Florida", "Georgia"]
   
   if state in west:
       return 1
   elif state in northeast:
       return 2
   elif state in east:
       return 3
   elif state in south:
       return 4
   elif state in center:
       return 5
   elif state in north:
       return 6
   elif state in north_west:
       return 7
   elif state in south_east:
       return 8
   else:
       return 9 



df_prep['What US state or territory do you work in?'] = df_prep['What US state or territory do you work in?'].apply(create_state_encoding)



df_prep['What US state or territory do you work in?'].value_counts()



df_prep['Are you self-employed?'].value_counts()


# As all values in 'Are you self-employed?' column are zero, I shall eliminate this column
df_prep.drop('Are you self-employed?',  axis=1, inplace=True)



df['Do you know the options for mental health care available under your employer-provided coverage?'].value_counts(dropna = False)


df_prep['Do you know the options for mental health care available under your employer-provided coverage?'].fillna('I am not sure',inplace = True)


df['Do you know the options for mental health care available under your employer-provided coverage?'].value_counts(dropna = False)


df['Does your employer provide mental health benefits as part of healthcare coverage?'].value_counts(dropna = False)


df['Do you know the options for mental health care available under your employer-provided coverage?'].value_counts(dropna = False)


df['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'].value_counts(dropna = False)


df['Does your employer offer resources to learn more about mental health concerns and options for seeking help?'].value_counts(dropna = False)


df['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'].value_counts(dropna = False)



df['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'].value_counts(dropna = False)


df['Do you think that discussing a mental health disorder with your employer would have negative consequences?'].value_counts(dropna = False)


df['Do you think that discussing a physical health issue with your employer would have negative consequences?'].value_counts(dropna = False)


df['Would you feel comfortable discussing a mental health disorder with your coworkers?'].value_counts(dropna = False)


df['Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?'].value_counts(dropna = False)


df['Do you feel that your employer takes mental health as seriously as physical health?'].value_counts(dropna = False)


df['Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'].value_counts(dropna = False)


df['Do you have previous employers?'].value_counts(dropna = False)


df['Would you be willing to bring up a physical health issue with a potential employer in an interview?'].value_counts(dropna = False)


df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].value_counts(dropna = False)


df_prep['Do you feel that being identified as a person with a mental health issue would hurt your career?'] = df_prep['Do you feel that being identified as a person with a mental health issue would hurt your career?'].map({'Yes, I think it would': 'Yes', "No, I don't think it would":'No','Yes, it has': 'Yes', "No, it has not":'No','Maybe':'Maybe'})



df['Do you feel that being identified as a person with a mental health issue would hurt your career?'].value_counts(dropna = False)


df['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].value_counts(dropna = False)


df_prep['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'] = df_prep['Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?'].map({"No, I don't think they would": 'No','Maybe':'Maybe','Yes, they do':'Yes','Yes, I think they would':'Yes','No, they do not':'No'})


df['How willing would you be to share with friends and family that you have a mental illness?'].value_counts(dropna = False)


df['How willing would you be to share with friends and family that you have a mental illness?'].value_counts(dropna = False)


df['Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?'].value_counts(dropna = False)


df_prep['Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?'] = df_prep['Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?'].map({'No': 'No','Maybe/Not sure':'Maybe','Yes, I experienced':'Yes','Yes, I observed':'Yes'})


df['Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?'].value_counts(dropna = False)


df_prep['Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?'].fillna('I am not sure',inplace = True)


df['Do you have a family history of mental illness?'].value_counts(dropna = False)


df['Have you had a mental health disorder in the past?'].value_counts(dropna = False)


df['Do you currently have a mental health disorder?'].value_counts(dropna = False)


df['Have you been diagnosed with a mental health condition by a medical professional?'].value_counts(dropna = False)


df['Have you ever sought treatment for a mental health issue from a mental health professional?'].value_counts(dropna = False)


df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].value_counts(dropna = False)


df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].value_counts(dropna = False)


df['What US state or territory do you work in?'].value_counts(dropna = False)


df['Do you work remotely?'].value_counts(dropna = False)


df['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'].value_counts()


ordinal_orderings = [
    ["I don't know", 'Very easy', 'Somewhat easy', 'Neither easy nor difficult','Somewhat difficult','Very difficult']
]
ordinal_features = [
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'
]
def ordinal_encode_v1 (df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: (ordering.index(x)))
    return df
df_prep = ordinal_encode_v1(
    df_prep, columns=ordinal_features,orderings=ordinal_orderings
)

df_prep.isnull().sum().sum()
df_prep.to_csv('C:/Users/Dell/Documents/book/unsuper/send for github/cleaned_data_for_prediction.csv')







0
Median Age is : 32.0
(1146, 60)
Have your previous employers provided mental health benefits?
Some did             326
No, none did         280
I don't know         258
Yes, they all did    151
NaN                  131
Name: count, dtype: int64
